 # this file will try to trian the two models defined here. And determine which is the best performing one

In [19]:
from torch.nn import Module, Sequential, LeakyReLU, Conv2d, BatchNorm2d, AvgPool2d, MaxPool2d, AdaptiveAvgPool2d, Linear, Dropout
import torch
from torchinfo import summary

# this is adapted from https://github.com/Moeo3/GoogLeNet-Inception-V3-pytorch/blob/master/googlenet_v3.py#L58 with its size modified to suit the CIFAR10 dataset instead of the origianl ImageNet dataset.

''' comments on models
the orignal model has
3 conv_bn
1 pool
2 conv_bn
1 pool
3x inception a
1x inception b
4x inception c
1x inception d
2x inception e
1 conv_bn
1 adaptive pool 2d

dropout
flatten

fully connected layer

===================================
for our model, we are gonna just makeit smaller so that it trains faster, also cifar10 does not have the 1000 classes in imagenet lol
'''

class InceptionModel1(Module):
    def __init__(self, channels_in, class_num = 10):
        super(InceptionModel1, self).__init__()
        # remember, i must be able to extract the feature maps of each of the convolution layers, and as such, i must design my network around that as well

        # if this one is false, it will return feature maps. This would be found at the return funtion in the forward function
        self.PCA = False
        # input is N, 3, 32, 32
        self.layer1 = Sequential(
            Conv2d_BN(channels_in = channels_in, channels_out= 32, kernel_size=3, stride=2, padding=1),
            MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1)
             )
        # N, 32, 16, 16
        self.layer2 = Sequential(
            Conv2d_BN(channels_in = 32, channels_out= 64, kernel_size=3, stride=2, padding=1),
            MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1)
        )
        # N, 64, 8, 8

        # going into the inception layers
        # note that each of the components inside inception will ALWAYS retain the same width and height, and all the channels are concatenated together thats all
        self.incep1 = InceptionA(64, 16)
        # N, 240, 8, 8
        self.incep2 = InceptionB(240)
        # N, 720, 8, 8
        # inception 3 barely fits CIFAR10, i think this will be the last layer
        self.incep3 = InceptionC(720, 128)
        # N, 768, 8, 8
        self.incep4 = InceptionD(768)
        # N, 1280, 8, 8
        self.incep5 = InceptionE(1280)
        # N, 2048, 8, 8

        # going into the output layer now, last conv layer and then flattening it
        self.out = Sequential(
            # lowering the number of channels
            Conv2d_BN(channels_in = 2048, channels_out= 1024, kernel_size=1, stride=1, padding=0),
            AdaptiveAvgPool2d(1),
            Dropout(0.5)
        )
        # N, 1024, 1, 1

        # this one will then output it based on the number of classes, based on softmax i guess at this point
        self.fc = Linear(1024, class_num)

    def forward(self, x):
        x = self.layer1(x)
        fmap1 = x.clone()
        x = self.layer2(x)
        fmap2 = x.clone()
        x = self.incep1(x)
        fmap3 = x.clone()
        x = self.incep2(x)
        fmap4 = x.clone()
        x = self.incep3(x)
        fmap5 = x.clone()
        x = self.incep4(x)
        fmap6 = x.clone()
        x = self.incep5(x)
        fmap7 = x.clone()
        x = self.out(x)
        fmap8 = x.clone()
        # this one is to flatten, and retaining the batch size
        x = torch.flatten(x, 1)
        x = self.fc(x)
        fmap9 = x.clone()

        if self.PCA:
            return x, (fmap1, fmap2, fmap3, fmap3, fmap4, fmap5, fmap6, fmap7, fmap8, fmap9)
        else:
            return x

class InceptionModel2(Module):
    def __init__(self, channels_in, class_num = 10):
        super(InceptionModel2, self).__init__()
        # remember, i must be able to extract the feature maps of each of the convolution layers, and as such, i must design my network around that as well

        # if this one is false, it will return feature maps. This would be found at the return funtion in the forward function
        self.PCA = False
        # input is N, 3, 32, 32
        self.layer1 = Sequential(
            Conv2d_BN(channels_in = channels_in, channels_out= 32, kernel_size=3, stride=1, padding=1),
            MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1)
        )
        # N, 32, 16, 16
        self.layer2 = Sequential(
            Conv2d_BN(channels_in = 32, channels_out= 64, kernel_size=3, stride=1, padding=1),
            MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1)
        )
        # N, 64, 8, 8

        # going into the inception layers
        # note that each of the components inside inception will ALWAYS retain the same width and height, and all the channels are concatenated together thats all
        self.incep1 = InceptionA(64, 16)
        # N, 240, 8, 8
        self.incep2 = InceptionB(240)
        # N, 720, 8, 8
        # inception 3 barely fits CIFAR10, i think this will be the last layer
        self.incep3 = InceptionC(720, 128)
        # N, 768, 8, 8

        # going into the output layer now, last conv layer and then flattening it
        self.out = Sequential(
            # lowering the number of channels
            Conv2d_BN(channels_in = 768, channels_out= 320, kernel_size=1, stride=1, padding=0),
            AdaptiveAvgPool2d(1),
            Dropout(0.5)
        )
        # N, 1024, 1, 1

        # this one will then output it based on the number of classes, based on softmax i guess at this point
        self.fc = Linear(320, class_num)

    def forward(self, x):
        x = self.layer1(x)
        fmap1 = x.clone()
        x = self.layer2(x)
        fmap2 = x.clone()
        x = self.incep1(x)
        fmap3 = x.clone()
        x = self.incep2(x)
        fmap4 = x.clone()
        x = self.incep3(x)
        fmap5 = x.clone()
        x = self.out(x)
        fmap8 = x.clone()
        # this one is to flatten, and retaining the batch size
        x = torch.flatten(x, 1)
        x = self.fc(x)
        fmap9 = x.clone()

        if self.PCA:
            return x, (fmap1, fmap2, fmap3, fmap3, fmap4, fmap5, fmap8, fmap9)
        else:
            return x


# this is the intermediate modules
class Conv2d_BN(Module):
    def __init__(self, channels_in, channels_out, kernel_size, padding, stride=1, acti=LeakyReLU(0.2, inplace=True)):
        super(Conv2d_BN, self).__init__()
        self.conv2d_bn = Sequential(
            Conv2d(channels_in, channels_out, kernel_size, stride, padding, bias=False),
            BatchNorm2d(channels_out),
            acti
        )

    def forward(self, x):
        return self.conv2d_bn(x)

class InceptionA(Module):
    def __init__(self, channels_in, pool_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = Conv2d_BN(channels_in, 64, 1, stride=1, padding=0)  # 64 channels
        self.branch5x5 = Sequential(
            Conv2d_BN(channels_in, 48, 1, stride=1, padding=0),
            Conv2d_BN(48, 64, 5, stride=1, padding=2)
        )  # 64 channels
        self.branch3x3dbl = Sequential(
            Conv2d_BN(channels_in, 64, 1, stride=1, padding=0),
            Conv2d_BN(64, 96, 3, stride=1, padding=1),
            Conv2d_BN(96, 96, 3, stride=1, padding=1)
        )  # 96 channels
        self.branch_pool = Sequential(
            AvgPool2d(3, stride=1, padding=1),
            Conv2d_BN(channels_in, pool_channels, 1, stride=1, padding=0)
        )  # pool_channels

    def forward(self, x):
        outputs = [self.branch1x1(x), self.branch5x5(x), self.branch3x3dbl(x), self.branch_pool(x)]
        # 64 + 64 + 96 + pool_channels
        return torch.cat(outputs, 1)

class InceptionB(Module):
    def __init__(self, channels_in):
        super(InceptionB, self).__init__()
        self.branch3x3 = Conv2d_BN(channels_in, 384, 3, stride=2, padding=1)  # 384 channels
        self.branch3x3dbl = Sequential(
            Conv2d_BN(channels_in, 64, 1, padding=0),
            Conv2d_BN(64, 96, 3, padding=1),
            Conv2d_BN(96, 96, 3, stride=2, padding=1)
        )  # 96 channels
        self.branch_pool = MaxPool2d(3, stride=2, padding=1)  # channels_in

    def forward(self, x):
        outputs = [self.branch3x3(x), self.branch3x3dbl(x), self.branch_pool(x)]
        # 384 + 96 + channels_in
        return torch.cat(outputs, 1)

class InceptionC(Module):
    def __init__(self, channels_in, channels_7x7):
        super(InceptionC, self).__init__()
        self.branch1x1 = Conv2d_BN(channels_in, 192, 1, stride=1, padding=0)  # 192 channels
        self.branch7x7 = Sequential(
            Conv2d_BN(channels_in, channels_7x7, 1, stride=1, padding=0),
            Conv2d_BN(channels_7x7, channels_7x7, (1, 7), stride=1, padding=(0, 3)),
            Conv2d_BN(channels_7x7, 192, (7, 1), stride=1, padding=(3, 0))
        )  # 192 channels
        self.branch7x7dbl = Sequential(
            Conv2d_BN(channels_in, channels_7x7, 1, stride=1, padding=0),
            Conv2d_BN(channels_7x7, channels_7x7, (7, 1), stride=1, padding=(3, 0)),
            Conv2d_BN(channels_7x7, channels_7x7, (1, 7), stride=1, padding=(0, 3)),
            Conv2d_BN(channels_7x7, channels_7x7, (7, 1), stride=1, padding=(3, 0)),
            Conv2d_BN(channels_7x7, 192, (1, 7), stride=1, padding=(0, 3))
        )  # 192 channels
        self.branch_pool = Sequential(
            AvgPool2d(3, stride=1, padding=1),
            Conv2d_BN(channels_in, 192, 1, stride=1, padding=0)
        )  # 192 channels

    def forward(self, x):
        outputs = [self.branch1x1(x), self.branch7x7(x), self.branch7x7dbl(x), self.branch_pool(x)]
        # 192 + 192 + 192 + 192 = 768 channels
        return torch.cat(outputs, 1)

class InceptionD(Module):
    def __init__(self, channels_in):
        super(InceptionD, self).__init__()
        self.branch3x3 = Sequential(
            Conv2d_BN(channels_in, 192, 1, stride=1, padding=0),
            Conv2d_BN(192, 320, 3, stride=2, padding=1)
        )  # 320 channels
        self.branch7x7x3 = Sequential(
            Conv2d_BN(channels_in, 192, 1, stride=1, padding=0),
            Conv2d_BN(192, 192, (1, 7), stride=1, padding=(0, 3)),
            Conv2d_BN(192, 192, (7, 1), stride=1, padding=(3, 0)),
            Conv2d_BN(192, 192, 3, stride=2, padding=1)
        )  # 192 chnnels
        self.branch_pool = MaxPool2d(3, stride=2, padding=1)  # channels_in

    def forward(self, x):
        outputs = [self.branch3x3(x), self.branch7x7x3(x), self.branch_pool(x)]
        # 320 + 192 + channels_in
        return torch.cat(outputs, 1)

class InceptionE(Module):
    def __init__(self, channels_in):
        super(InceptionE, self).__init__()
        self.branch1x1 = Conv2d_BN(channels_in, 320, 1, stride=1, padding=0)  # 320 channels

        self.branch3x3_1 = Conv2d_BN(channels_in, 384, 1, stride=1, padding=0)
        self.branch3x3_2a = Conv2d_BN(384, 384, (1, 3), stride=1, padding=(0, 1))
        self.branch3x3_2b = Conv2d_BN(384, 384, (3, 1), stride=1, padding=(1, 0))
        # 768 channels

        self.branch3x3dbl_1 = Sequential(
            Conv2d_BN(channels_in, 448, 1, stride=1, padding=0),
            Conv2d_BN(448, 384, 3, stride=1, padding=1)
        )
        self.branch3x3dbl_2a = Conv2d_BN(384, 384, (1, 3), stride=1, padding=(0, 1))
        self.branch3x3dbl_2b = Conv2d_BN(384, 384, (3, 1), stride=1, padding=(1, 0))
        # 768 channels

        self.branch_pool = Sequential(
            AvgPool2d(3, stride=1, padding=1),
            Conv2d_BN(channels_in, 192, 1, stride=1, padding=0)
        )  # 192 channels
    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = torch.cat([self.branch3x3_2a(branch3x3), self.branch3x3_2b(branch3x3)], 1)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = torch.cat([self.branch3x3dbl_2a(branch3x3dbl), self.branch3x3dbl_2b(branch3x3dbl)], 1)

        branch_pool = self.branch_pool(x)

        outputs = [branch1x1, branch3x3, branch3x3dbl, branch_pool]
        # 320 + 768 + 768 + 192 = 2048 channels
        return torch.cat(outputs, 1)

In [20]:
# model = InceptionModel1(channels_in=3, class_num=10)
# print(summary(model, (64, 3, 32, 32)))

In [21]:
model = InceptionModel2(channels_in=3, class_num=10)
print(summary(model, (64, 3, 32, 32)))

Layer (type:depth-idx)                        Output Shape              Param #
InceptionModel2                               [64, 10]                  --
├─Sequential: 1-1                             [64, 32, 16, 16]          --
│    └─Conv2d_BN: 2-1                         [64, 32, 32, 32]          --
│    │    └─Sequential: 3-1                   [64, 32, 32, 32]          928
├─Sequential: 1-49                            --                        (recursive)
│    └─Conv2d_BN: 2-49                        --                        (recursive)
│    │    └─Sequential: 3-48                  --                        (recursive)
├─Sequential: 1-3                             --                        (recursive)
│    └─MaxPool2d: 2-3                         [64, 32, 16, 16]          --
├─Sequential: 1-4                             [64, 64, 8, 8]            --
│    └─Conv2d_BN: 2-4                         [64, 64, 16, 16]          --
│    │    └─Sequential: 3-3                   [64, 64, 16,

In [22]:

model.PCA = True
a, b = model.forward(torch.randn(64, 3, 32, 32).to('cuda'))

In [3]:
STUDENTID = 567     # this will be used for random states

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import logging
from utils import *
import torch.optim as optim
import os
import shutil
import os
import time

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    # this normalization is industry standard, it seems
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize the data
])


# define the label transformations, from int64 to float32
transform_label = transforms.Compose([
    #transforms.ToTensor()
])

# Download and load CIFAR-100 datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True, target_transform=transform_label)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True, target_transform=transform_label)


# Calculate the sizes for train, validation, and test sets
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

# Split the train dataset into train and validation sets using a random seed
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(STUDENTID))

# Create data loaders for training, validation, and test sets
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print dataset sizes
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(valid_dataset))
print("Number of test examples:", len(test_dataset))

# creating a model that automatically runs the forward function i guess, since it is easier
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
from customModels import *

Files already downloaded and verified
Files already downloaded and verified
Number of training examples: 40000
Number of validation examples: 10000
Number of test examples: 10000


In [13]:
model = InceptionModel1(channels_in=3, class_num=10)
modelName = 'InceptionModel1_corect_data_aug'

argDict = {
    'lr': 0.001,
    'maxEpoch': 250,
    'idleEpoch': 25,
    'outputName': modelName,
    'optimizer': optim.SGD(model.parameters(), lr=0.001),
    'criterion': nn.CrossEntropyLoss()
}

# setting up the logger
loggerName = modelName + '.log'
loggerName = os.path.join(argDict['outputName'], loggerName)
logger = MyLogger(loggerName)
argDict['logger'] = logger

# just to initilalize the files
logger.log('training starts here')

start_time = time.time()

# training and saving model to dictionary
outputDict = train(model, argDict, train_loader, val_loader, test_loader)

# loading the best model, and then sending it off to testing
model = load_model_from_file(model, argDict['outputName'], argDict['outputName'])

test_accuracy = test(model, argDict, test_loader)
tempString = 'testing accuracy of ' + argDict['outputName'] + " is: " + str(test_accuracy)
logger.log(tempString)

argDict['test_accuracy'] = str(test_accuracy)

# timing the thing as well
end_time = time.time()
execution_time = end_time - start_time
argDict['time_taken'] = execution_time
save_dict_to_file(outputDict, argDict['outputName'], argDict['outputName'])

del model
del argDict

# Define the folder you want to zip and the output zip file name
folder_to_zip = modelName
output_zip_file = modelName + ".zip"

# Use shutil.make_archive to create the zip file
shutil.make_archive(output_zip_file, 'zip', folder_to_zip)

os.rename(output_zip_file + '.zip', output_zip_file)

training starts here
currently at epoch 0 train accuracy: tensor(0.3019, device='cuda:0') loss of: 1.911883002090454 eval accuracy: tensor(0.3727, device='cuda:0')
currently at epoch 1 train accuracy: tensor(0.4021, device='cuda:0') loss of: 1.6478466848373412 eval accuracy: tensor(0.4132, device='cuda:0')
currently at epoch 2 train accuracy: tensor(0.4464, device='cuda:0') loss of: 1.5311889154434204 eval accuracy: tensor(0.4423, device='cuda:0')
currently at epoch 3 train accuracy: tensor(0.4740, device='cuda:0') loss of: 1.4532343326568604 eval accuracy: tensor(0.4566, device='cuda:0')
currently at epoch 4 train accuracy: tensor(0.5013, device='cuda:0') loss of: 1.3910504089355469 eval accuracy: tensor(0.4704, device='cuda:0')
currently at epoch 5 train accuracy: tensor(0.5180, device='cuda:0') loss of: 1.3425525281906128 eval accuracy: tensor(0.4737, device='cuda:0')
currently at epoch 6 train accuracy: tensor(0.5350, device='cuda:0') loss of: 1.2994744129180908 eval accuracy: tens

In [14]:
model = InceptionModel2(channels_in=3, class_num=10)
modelName = 'InceptionModel2_corect_data_aug'

argDict = {
    'lr': 0.001,
    'maxEpoch': 250,
    'idleEpoch': 25,
    'outputName': modelName,
    'optimizer': optim.SGD(model.parameters(), lr=0.001),
    'criterion': nn.CrossEntropyLoss()
}

# setting up the logger
loggerName = modelName + '.log'
loggerName = os.path.join(argDict['outputName'], loggerName)
logger = MyLogger(loggerName)
argDict['logger'] = logger

# just to initilalize the files
logger.log('training starts here')

start_time = time.time()

# training and saving model to dictionary
outputDict = train(model, argDict, train_loader, val_loader, test_loader)

# loading the best model, and then sending it off to testing
model = load_model_from_file(model, argDict['outputName'], argDict['outputName'])

test_accuracy = test(model, argDict, test_loader)
tempString = 'testing accuracy of ' + argDict['outputName'] + " is: " + str(test_accuracy)
logger.log(tempString)

argDict['test_accuracy'] = str(test_accuracy)

# timing the thing as well
end_time = time.time()
execution_time = end_time - start_time
argDict['time_taken'] = execution_time
save_dict_to_file(outputDict, argDict['outputName'], argDict['outputName'])

del model
del argDict

# Define the folder you want to zip and the output zip file name
folder_to_zip = modelName
output_zip_file = modelName + ".zip"

# Use shutil.make_archive to create the zip file
shutil.make_archive(output_zip_file, 'zip', folder_to_zip)

os.rename(output_zip_file + '.zip', output_zip_file)

training starts here
currently at epoch 0 train accuracy: tensor(0.2759, device='cuda:0') loss of: 1.9911288063049317 eval accuracy: tensor(0.3407, device='cuda:0')
currently at epoch 1 train accuracy: tensor(0.3696, device='cuda:0') loss of: 1.732855928993225 eval accuracy: tensor(0.3887, device='cuda:0')
currently at epoch 2 train accuracy: tensor(0.4188, device='cuda:0') loss of: 1.6181901485443115 eval accuracy: tensor(0.4180, device='cuda:0')
currently at epoch 3 train accuracy: tensor(0.4447, device='cuda:0') loss of: 1.5457998191833495 eval accuracy: tensor(0.4387, device='cuda:0')
currently at epoch 4 train accuracy: tensor(0.4642, device='cuda:0') loss of: 1.4877448209762574 eval accuracy: tensor(0.4482, device='cuda:0')
currently at epoch 5 train accuracy: tensor(0.4836, device='cuda:0') loss of: 1.4421364206314087 eval accuracy: tensor(0.4639, device='cuda:0')
currently at epoch 6 train accuracy: tensor(0.4977, device='cuda:0') loss of: 1.4030528705596923 eval accuracy: tens

In [15]:
model = InceptionModel2(channels_in=3, class_num=10)
modelName = 'InceptionModel2_data_aug_cropping'

# this transformation sequence must be modified to work with the random stuff
# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    # this normalization is industry standard, it seems
    transforms.RandomCrop(size=28, padding=2, pad_if_needed=True, padding_mode="edge"),
    transforms.Resize(32),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize the data
])

argDict = {
    'lr': 0.001,
    'maxEpoch': 250,
    'idleEpoch': 25,
    'outputName': modelName,
    'optimizer': optim.SGD(model.parameters(), lr=0.001),
    'criterion': nn.CrossEntropyLoss()
}

STUDENTID = 567     # this will be used for random states

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import logging
from utils import *
import torch.optim as optim
import os
import shutil
import os
import time


# define the label transformations, from int64 to float32
transform_label = transforms.Compose([
    #transforms.ToTensor()
])

# Download and load CIFAR-100 datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True, target_transform=transform_label)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True, target_transform=transform_label)


# Calculate the sizes for train, validation, and test sets
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

# Split the train dataset into train and validation sets using a random seed
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(STUDENTID))

# Create data loaders for training, validation, and test sets
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print dataset sizes
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(valid_dataset))
print("Number of test examples:", len(test_dataset))

# creating a model that automatically runs the forward function i guess, since it is easier
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
from customModels import *

# setting up the logger
loggerName = modelName + '.log'
loggerName = os.path.join(argDict['outputName'], loggerName)
logger = MyLogger(loggerName)
argDict['logger'] = logger

# just to initilalize the files
logger.log('training starts here')

start_time = time.time()

# training and saving model to dictionary
outputDict = train(model, argDict, train_loader, val_loader, test_loader)

# loading the best model, and then sending it off to testing
model = load_model_from_file(model, argDict['outputName'], argDict['outputName'])

test_accuracy = test(model, argDict, test_loader)
tempString = 'testing accuracy of ' + argDict['outputName'] + " is: " + str(test_accuracy)
logger.log(tempString)

argDict['test_accuracy'] = str(test_accuracy)

# timing the thing as well
end_time = time.time()
execution_time = end_time - start_time
argDict['time_taken'] = execution_time
save_dict_to_file(outputDict, argDict['outputName'], argDict['outputName'])

del model
del argDict

# Define the folder you want to zip and the output zip file name
folder_to_zip = modelName
output_zip_file = modelName + ".zip"

# Use shutil.make_archive to create the zip file
shutil.make_archive(output_zip_file, 'zip', folder_to_zip)

os.rename(output_zip_file + '.zip', output_zip_file)

Files already downloaded and verified
Files already downloaded and verified
Number of training examples: 40000
Number of validation examples: 10000
Number of test examples: 10000
training starts here
currently at epoch 0 train accuracy: tensor(0.2651, device='cuda:0') loss of: 1.9998745420455932 eval accuracy: tensor(0.3115, device='cuda:0')
currently at epoch 1 train accuracy: tensor(0.3394, device='cuda:0') loss of: 1.7993052368164062 eval accuracy: tensor(0.3529, device='cuda:0')
currently at epoch 2 train accuracy: tensor(0.3741, device='cuda:0') loss of: 1.717935894393921 eval accuracy: tensor(0.3789, device='cuda:0')
currently at epoch 3 train accuracy: tensor(0.3968, device='cuda:0') loss of: 1.662701590538025 eval accuracy: tensor(0.3962, device='cuda:0')
currently at epoch 4 train accuracy: tensor(0.4097, device='cuda:0') loss of: 1.6310093156814576 eval accuracy: tensor(0.4178, device='cuda:0')
currently at epoch 5 train accuracy: tensor(0.4251, device='cuda:0') loss of: 1.59

In [16]:
model = InceptionModel2(channels_in=3, class_num=10)
modelName = 'InceptionModel2_data_aug_flip'

# this transformation sequence must be modified to work with the random stuff
# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    # this normalization is industry standard, it seems
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.Resize(32),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize the data
])

argDict = {
    'lr': 0.001,
    'maxEpoch': 250,
    'idleEpoch': 25,
    'outputName': modelName,
    'optimizer': optim.SGD(model.parameters(), lr=0.001),
    'criterion': nn.CrossEntropyLoss()
}

STUDENTID = 567     # this will be used for random states

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import logging
from utils import *
import torch.optim as optim
import os
import shutil
import os
import time


# define the label transformations, from int64 to float32
transform_label = transforms.Compose([
    #transforms.ToTensor()
])

# Download and load CIFAR-100 datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True, target_transform=transform_label)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True, target_transform=transform_label)


# Calculate the sizes for train, validation, and test sets
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

# Split the train dataset into train and validation sets using a random seed
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(STUDENTID))

# Create data loaders for training, validation, and test sets
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print dataset sizes
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(valid_dataset))
print("Number of test examples:", len(test_dataset))

# creating a model that automatically runs the forward function i guess, since it is easier
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
from customModels import *

# setting up the logger
loggerName = modelName + '.log'
loggerName = os.path.join(argDict['outputName'], loggerName)
logger = MyLogger(loggerName)
argDict['logger'] = logger

# just to initilalize the files
logger.log('training starts here')

start_time = time.time()

# training and saving model to dictionary
outputDict = train(model, argDict, train_loader, val_loader, test_loader)

# loading the best model, and then sending it off to testing
model = load_model_from_file(model, argDict['outputName'], argDict['outputName'])

test_accuracy = test(model, argDict, test_loader)
tempString = 'testing accuracy of ' + argDict['outputName'] + " is: " + str(test_accuracy)
logger.log(tempString)

argDict['test_accuracy'] = str(test_accuracy)

# timing the thing as well
end_time = time.time()
execution_time = end_time - start_time
argDict['time_taken'] = execution_time
save_dict_to_file(outputDict, argDict['outputName'], argDict['outputName'])

del model
del argDict

# Define the folder you want to zip and the output zip file name
folder_to_zip = modelName
output_zip_file = modelName + ".zip"

# Use shutil.make_archive to create the zip file
shutil.make_archive(output_zip_file, 'zip', folder_to_zip)

os.rename(output_zip_file + '.zip', output_zip_file)

Files already downloaded and verified
Files already downloaded and verified
Number of training examples: 40000
Number of validation examples: 10000
Number of test examples: 10000
training starts here
currently at epoch 0 train accuracy: tensor(0.2536, device='cuda:0') loss of: 2.039087518501282 eval accuracy: tensor(0.3123, device='cuda:0')
currently at epoch 1 train accuracy: tensor(0.3392, device='cuda:0') loss of: 1.8063823118209839 eval accuracy: tensor(0.3582, device='cuda:0')
currently at epoch 2 train accuracy: tensor(0.3709, device='cuda:0') loss of: 1.7219138929367066 eval accuracy: tensor(0.3760, device='cuda:0')
currently at epoch 3 train accuracy: tensor(0.3946, device='cuda:0') loss of: 1.6668205379486083 eval accuracy: tensor(0.3927, device='cuda:0')
currently at epoch 4 train accuracy: tensor(0.4090, device='cuda:0') loss of: 1.6224696214675904 eval accuracy: tensor(0.4104, device='cuda:0')
currently at epoch 5 train accuracy: tensor(0.4232, device='cuda:0') loss of: 1.5

In [17]:
model = InceptionModel2(channels_in=3, class_num=10)
modelName = 'InceptionModel2_data_aug_rotation'

# this transformation sequence must be modified to work with the random stuff
# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    # this normalization is industry standard, it seems
    transforms.RandomRotation(360),
    transforms.Resize(32),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize the data
])

argDict = {
    'lr': 0.001,
    'maxEpoch': 250,
    'idleEpoch': 25,
    'outputName': modelName,
    'optimizer': optim.SGD(model.parameters(), lr=0.001),
    'criterion': nn.CrossEntropyLoss()
}

STUDENTID = 567     # this will be used for random states

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import logging
from utils import *
import torch.optim as optim
import os
import shutil
import os
import time


# define the label transformations, from int64 to float32
transform_label = transforms.Compose([
    #transforms.ToTensor()
])

# Download and load CIFAR-100 datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True, target_transform=transform_label)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True, target_transform=transform_label)


# Calculate the sizes for train, validation, and test sets
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

# Split the train dataset into train and validation sets using a random seed
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(STUDENTID))

# Create data loaders for training, validation, and test sets
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print dataset sizes
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(valid_dataset))
print("Number of test examples:", len(test_dataset))

# creating a model that automatically runs the forward function i guess, since it is easier
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
from customModels import *

# setting up the logger
loggerName = modelName + '.log'
loggerName = os.path.join(argDict['outputName'], loggerName)
logger = MyLogger(loggerName)
argDict['logger'] = logger

# just to initilalize the files
logger.log('training starts here')

start_time = time.time()

# training and saving model to dictionary
outputDict = train(model, argDict, train_loader, val_loader, test_loader)

# loading the best model, and then sending it off to testing
model = load_model_from_file(model, argDict['outputName'], argDict['outputName'])

test_accuracy = test(model, argDict, test_loader)
tempString = 'testing accuracy of ' + argDict['outputName'] + " is: " + str(test_accuracy)
logger.log(tempString)

argDict['test_accuracy'] = str(test_accuracy)

# timing the thing as well
end_time = time.time()
execution_time = end_time - start_time
argDict['time_taken'] = execution_time
save_dict_to_file(outputDict, argDict['outputName'], argDict['outputName'])

del model
del argDict

# Define the folder you want to zip and the output zip file name
folder_to_zip = modelName
output_zip_file = modelName + ".zip"

# Use shutil.make_archive to create the zip file
shutil.make_archive(output_zip_file, 'zip', folder_to_zip)

os.rename(output_zip_file + '.zip', output_zip_file)

Files already downloaded and verified
Files already downloaded and verified
Number of training examples: 40000
Number of validation examples: 10000
Number of test examples: 10000
training starts here
currently at epoch 0 train accuracy: tensor(0.2351, device='cuda:0') loss of: 2.0827041969299316 eval accuracy: tensor(0.2755, device='cuda:0')
currently at epoch 1 train accuracy: tensor(0.2947, device='cuda:0') loss of: 1.9207049633026123 eval accuracy: tensor(0.3028, device='cuda:0')
currently at epoch 2 train accuracy: tensor(0.3159, device='cuda:0') loss of: 1.859469598388672 eval accuracy: tensor(0.3275, device='cuda:0')
currently at epoch 3 train accuracy: tensor(0.3316, device='cuda:0') loss of: 1.8144521184921265 eval accuracy: tensor(0.3394, device='cuda:0')
currently at epoch 4 train accuracy: tensor(0.3460, device='cuda:0') loss of: 1.7829607051849365 eval accuracy: tensor(0.3558, device='cuda:0')
currently at epoch 5 train accuracy: tensor(0.3600, device='cuda:0') loss of: 1.7

In [4]:
model = InceptionModel2(channels_in=3, class_num=10)
modelName = 'InceptionModel2_data_aug_reiszeCrop'

# this transformation sequence must be modified to work with the random stuff
# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    # this normalization is industry standard, it seems
    transforms.RandomResizedCrop(size=32),
    transforms.Resize(32),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize the data
])

argDict = {
    'lr': 0.001,
    'maxEpoch': 250,
    'idleEpoch': 25,
    'outputName': modelName,
    'optimizer': optim.SGD(model.parameters(), lr=0.001),
    'criterion': nn.CrossEntropyLoss()
}

STUDENTID = 567     # this will be used for random states

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import logging
from utils import *
import torch.optim as optim
import os
import shutil
import os
import time


# define the label transformations, from int64 to float32
transform_label = transforms.Compose([
    #transforms.ToTensor()
])

# Download and load CIFAR-100 datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True, target_transform=transform_label)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True, target_transform=transform_label)


# Calculate the sizes for train, validation, and test sets
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

# Split the train dataset into train and validation sets using a random seed
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(STUDENTID))

# Create data loaders for training, validation, and test sets
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print dataset sizes
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(valid_dataset))
print("Number of test examples:", len(test_dataset))

# creating a model that automatically runs the forward function i guess, since it is easier
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
from customModels import *

# setting up the logger
loggerName = modelName + '.log'
loggerName = os.path.join(argDict['outputName'], loggerName)
logger = MyLogger(loggerName)
argDict['logger'] = logger

# just to initilalize the files
logger.log('training starts here')

start_time = time.time()

# training and saving model to dictionary
outputDict = train(model, argDict, train_loader, val_loader, test_loader)

# loading the best model, and then sending it off to testing
model = load_model_from_file(model, argDict['outputName'], argDict['outputName'])

test_accuracy = test(model, argDict, test_loader)
tempString = 'testing accuracy of ' + argDict['outputName'] + " is: " + str(test_accuracy)
logger.log(tempString)

argDict['test_accuracy'] = str(test_accuracy)

# timing the thing as well
end_time = time.time()
execution_time = end_time - start_time
argDict['time_taken'] = execution_time
save_dict_to_file(outputDict, argDict['outputName'], argDict['outputName'])

del model
del argDict

# Define the folder you want to zip and the output zip file name
folder_to_zip = modelName
output_zip_file = modelName + ".zip"

# Use shutil.make_archive to create the zip file
shutil.make_archive(output_zip_file, 'zip', folder_to_zip)

os.rename(output_zip_file + '.zip', output_zip_file)

Files already downloaded and verified
Files already downloaded and verified
Number of training examples: 40000
Number of validation examples: 10000
Number of test examples: 10000
training starts here
currently at epoch 0 train accuracy: tensor(0.2314, device='cuda:0') loss of: 2.078931097984314 eval accuracy: tensor(0.2701, device='cuda:0')
currently at epoch 1 train accuracy: tensor(0.2887, device='cuda:0') loss of: 1.9303240478515624 eval accuracy: tensor(0.3025, device='cuda:0')
currently at epoch 2 train accuracy: tensor(0.3093, device='cuda:0') loss of: 1.874745209121704 eval accuracy: tensor(0.3299, device='cuda:0')
currently at epoch 3 train accuracy: tensor(0.3272, device='cuda:0') loss of: 1.8353795532226562 eval accuracy: tensor(0.3334, device='cuda:0')
currently at epoch 4 train accuracy: tensor(0.3444, device='cuda:0') loss of: 1.7983158248901367 eval accuracy: tensor(0.3466, device='cuda:0')
currently at epoch 5 train accuracy: tensor(0.3518, device='cuda:0') loss of: 1.78